In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
import pylab
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import statsmodels.tsa.api as smt
import warnings
warnings.filterwarnings('ignore')

In [37]:
from scipy import stats
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
from sklearn.datasets import load_boston
boston=load_boston()
cols = boston.feature_names

bos = pd.DataFrame(boston.data, columns=cols)
bos['Price'] = boston.target

In [6]:
y = bos['Price']
X = bos.drop(['Price'],axis=1)
#xc = sm.add_constant(X)

In [13]:
pf = PolynomialFeatures()

x = pf.fit_transform(X)

list = pf.get_feature_names(X.columns)
x = pd.DataFrame(x, columns=list)

In [15]:
x.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 105 columns):
1                float64
CRIM             float64
ZN               float64
INDUS            float64
CHAS             float64
NOX              float64
RM               float64
AGE              float64
DIS              float64
RAD              float64
TAX              float64
PTRATIO          float64
B                float64
LSTAT            float64
CRIM^2           float64
CRIM ZN          float64
CRIM INDUS       float64
CRIM CHAS        float64
CRIM NOX         float64
CRIM RM          float64
CRIM AGE         float64
CRIM DIS         float64
CRIM RAD         float64
CRIM TAX         float64
CRIM PTRATIO     float64
CRIM B           float64
CRIM LSTAT       float64
ZN^2             float64
ZN INDUS         float64
ZN CHAS          float64
ZN NOX           float64
ZN RM            float64
ZN AGE           float64
ZN DIS           float64
ZN RAD           float64
ZN TAX           fl

In [16]:
lin_reg = sm.OLS(y,x)
result = lin_reg.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.929
Model:                            OLS   Adj. R-squared:                  0.911
Method:                 Least Squares   F-statistic:                     51.06
Date:                Tue, 22 Oct 2019   Prob (F-statistic):          1.19e-179
Time:                        15:00:23   Log-Likelihood:                -1171.0
No. Observations:                 506   AIC:                             2550.
Df Residuals:                     402   BIC:                             2990.
Df Model:                         103                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
1              -175.2819     85.173     -2.058

Backward  Elimination

In [25]:
cols = x.columns.to_list()
pmax = 1

while len(cols)>0 :
    X_1 = x[cols]
    model = sm.OLS(y, X_1).fit()
    p = pd.Series(model.pvalues.values, index=cols)
    pmax = max(p)
    feature_max_p = p.idxmax()
    if pmax>0.05 :
        cols.remove(feature_max_p)
    else:
        break
        
selected_features_BE = cols
print(selected_features_BE)

['1', 'INDUS', 'CHAS', 'RM', 'AGE', 'DIS', 'RAD', 'PTRATIO', 'CRIM CHAS', 'CRIM RM', 'CRIM TAX', 'CRIM LSTAT', 'ZN NOX', 'ZN RM', 'ZN TAX', 'ZN PTRATIO', 'INDUS^2', 'INDUS RM', 'INDUS B', 'CHAS^2', 'CHAS NOX', 'CHAS RM', 'CHAS PTRATIO', 'NOX DIS', 'NOX RAD', 'NOX TAX', 'NOX PTRATIO', 'NOX LSTAT', 'RM^2', 'RM AGE', 'RM TAX', 'RM PTRATIO', 'RM LSTAT', 'AGE RAD', 'AGE TAX', 'AGE B', 'AGE LSTAT', 'DIS^2', 'DIS RAD', 'DIS TAX', 'DIS LSTAT', 'RAD^2', 'RAD TAX', 'RAD LSTAT', 'TAX^2', 'TAX PTRATIO', 'TAX B', 'PTRATIO B', 'LSTAT^2']


In [26]:
x = x[selected_features_BE]
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.921
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     113.4
Date:                Tue, 22 Oct 2019   Prob (F-statistic):          6.11e-222
Time:                        15:18:26   Log-Likelihood:                -1198.5
No. Observations:                 506   AIC:                             2493.
Df Residuals:                     458   BIC:                             2696.
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
1              -67.3468     22.652     -2.973   

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [30]:
model = sm.OLS(y_train, x_train).fit()
print(model.rsquared)


0.933363085944006


In [39]:
y_pred = model.predict(x_test)

print('R2 Score: ', r2_score(y_test, y_pred))

mse = mean_squared_error(y_test, y_pred)
print("RMSE: ", np.sqrt(mse))

R2 Score:  0.8737719554304634
RMSE:  3.0668619708677567
